In [29]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
api_key = "DO NOT SHARE ON FILE"
baseurl = "http://api.weatherapi.com/v1"
city = "Pittsburgh"

In [31]:
today = pd.Timestamp.now().strftime("%Y-%m-%d")
astronomy_url = f"{baseurl}/astronomy.json?key={api_key}&q={city}&dt={today}"
astro_response = requests.get(astronomy_url)
astro_response.status_code

200

In [32]:
forecast_url = f"{baseurl}/forecast.json?key={api_key}&q={city}&days=3"
forecast_response = requests.get(forecast_url)
forecast_response.status_code

200

In [ ]:
current_url = f"{baseurl}/current.json?key={api_key}&q={city}&dt={today}"
current_response = requests.get(forecast_url)
current_response.status_code

200

In [44]:
def get_city_data(city):
    url_current = f"{baseurl}/current.json?key={api_key}&q={city}"
    url_astronomy = f"{baseurl}/astronomy.json?key={api_key}&q={city}&dt={today}"
    url_forecast = f"{baseurl}/forecast.json?key={api_key}&q={city}&days=3"

   #Current Weather Build
    current_response = requests.get(url_current)
    if current_response.status_code == 200:
        current_data = current_response.json()
        current = current_data['current']
        location = current_data['location']

        print(f"Weather for {location['name']}, {location['region']}")
        print(f"Temperature: {current['temp_f']} °F / {current['temp_c']} °C")
        print(f"Condition: {current['condition']['text']}")
        print(f"Humidity: {current['humidity']}%")
        print(f"Wind Speed: {current['wind_mph']} mph")
    else:
        print(f"No current weather data for {city}.")

    # Sunrise and Sunset Data
    astro_response = requests.get(url_astronomy)
    if astro_response.status_code == 200:
        astro_data = astro_response.json()
        astronomy = astro_data['astronomy']['astro']
        location = astro_data['location']

        print(f"Sunrise & Sunset for {location['name']}, {location['region']}")
        print(f"Sunrise: {astronomy['sunrise']}")
        print(f"Sunset: {astronomy['sunset']}")
        print(f"Moon Phase: {astronomy['moon_phase']}")
    else:
        print(f"No current data for Sunrise and Sunset for {city}.")

    # Forecast- 3 day
    forecast_response = requests.get(url_forecast)
    if forecast_response.status_code == 200:
        forecast_data = forecast_response.json()
        forecast_days = forecast_data['forecast']['forecastday']

        print(f"3-Day Forecast for {city}")
        for day in forecast_days:
            date_str = day['date']
            day_info = day['day']
            condition = day_info['condition']['text']
            max_temp = day_info['maxtemp_f']
            min_temp = day_info['mintemp_f']
            chance_of_rain = day_info.get('daily_chance_of_rain', 'N/A')

            print(f"{date_str}")
            print(f"Condition: {condition}")
            print(f"Max Temp: {max_temp} °F")
            print(f"Min Temp: {min_temp} °F")
            print(f"Chance of Rain: {chance_of_rain}%")
    else:
        print(f"Unable to predict 3 day forecast for {city}.")


In [45]:
get_city_data("Pitstbu")

No current weather data for Pitstbu.
No current data for Sunrise and Sunset for Pitstbu.
Unable to predict 3 day forecast for Pitstbu.


In [46]:
get_city_data("Pittsburgh")

Weather for Pittsburgh, Pennsylvania
Temperature: 82.0 °F / 27.8 °C
Condition: Partly cloudy
Humidity: 43%
Wind Speed: 8.3 mph
Sunrise & Sunset for Pittsburgh, Pennsylvania
Sunrise: 07:07 AM
Sunset: 07:18 PM
Moon Phase: New Moon
3-Day Forecast for Pittsburgh
2025-09-21
Condition: Sunny
Max Temp: 86.9 °F
Min Temp: 62.6 °F
Chance of Rain: 0%
2025-09-22
Condition: Moderate rain
Max Temp: 82.9 °F
Min Temp: 66.1 °F
Chance of Rain: 60%
2025-09-23
Condition: Heavy rain
Max Temp: 76.6 °F
Min Temp: 65.3 °F
Chance of Rain: 84%


In [71]:
cities = ["New York City", "Chicago", "Denver", "Los Angeles", "Honolulu"]
summary_data = []

for city in cities:
    url_current = f"{baseurl}/current.json?key={api_key}&q={city}"
    url_astronomy = f"{baseurl}/astronomy.json?key={api_key}&q={city}&dt={today}"
   
    response_current = requests.get(url_current)
    response_astronomy = requests.get(url_astronomy)

    if response_current.status_code == 200 and response_astronomy.status_code == 200:
        data_current=response_current.json()
        summary_data.append({
        "City": data_current['location']['name'], 
        "Tempature": data_current['current']['temp_f'],
        "Condition": data_current['current']['condition']['text'],
        "Humidity": data_current['current']['humidity'],
        "Wind Speed": data_current['current']['wind_mph'],
        "Sunrise": astro['sunrise'],
        "Sunset": astro['sunset'],
        "Moon Phase": astro['moon_phase']
        })




In [72]:
weather_summary_df = pd.DataFrame(summary_data)


In [73]:
weather_summary_df.head()

,City,Tempature,Condition,Humidity,Wind Speed,Sunrise,Sunset,Moon Phase
0,New York,69.1,Sunny,47,9.8,06:20 AM,06:28 PM,New Moon
1,Chicago,82.9,Partly cloudy,55,7.4,06:20 AM,06:28 PM,New Moon
2,Denver,80.4,Partly cloudy,22,4.7,06:20 AM,06:28 PM,New Moon
3,Los Angeles,73.9,Sunny,71,6.7,06:20 AM,06:28 PM,New Moon
4,Honolulu,84.7,Partly cloudy,55,19.0,06:20 AM,06:28 PM,New Moon


In [74]:
weather_summary_df.to_csv("sample_summary_data.csv", index=False)